In [ ]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, DensityMatrix, Pauli, Operator, random_statevector
from qiskit_aer import AerSimulator, Aer
import numpy as np
from scipy.linalg import sqrtm

In [ ]:
def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

In [ ]:
def generate_prob_lst(num_states):
    prob_lst = np.array([np.random.random() for _ in range(num_states)])
    prob_lst /= np.sum(prob_lst)
    return prob_lst

def get_rank(dm, tol=1e-10):
    return int(np.sum(np.linalg.eigvalsh(dm) > tol))

def get_fidelity(dm1, dm2, tol=1e-5):
    # assert is_legal(dm1) and is_legal(dm2), 'inputs are not legal density matrices'
    if not is_legal(dm1) and is_legal(dm2):
        print("Warning: inputs are not legal density matrices")
    fidelity = (np.trace(sqrtm(sqrtm(dm1) @ dm2 @ sqrtm(dm1)))) ** 2
    assert np.abs(np.imag(fidelity)) < tol, 'fidelity is not real within tol'
    return fidelity.real

def generate_dm(num_qubits, num_states, state_lst=None, prob_lst=None):
    if state_lst is None:
        state_lst = [random_statevector(2**num_qubits) for _ in range(num_states)]
    if prob_lst is None:
        prob_lst = generate_prob_lst(num_states)
    density_matrix = sum([DensityMatrix(state_lst[i]).data * prob_lst[i] for i in range(num_states)])
    return density_matrix

def generate_IXYZ_strings(num_strings, length):
    characters = ['I', 'X', 'Y', 'Z']
    generated_strings = []
    assert num_strings < 4 ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != 'I' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_Pauli_expectations(dm, obsv):
    return np.trace(dm @ Pauli(obsv).to_matrix()).real

def get_trace_norm(dm):
    return np.sum(np.linalg.svd(dm, compute_uv=False))

In [ ]:
# Prepare a state for measurement
state = generate_dm(6, 2, prob_lst=np.array([.99, .01]))
r = get_rank(state)
d = 2 ** 6

In [ ]:
# Prepare observables
c = 1
num_measurements = int(c * r * d * (np.log(d)) ** 2)
observables = generate_IXYZ_strings(num_measurements, 6)
expectations = [generate_Pauli_expectations(state, obsv) for obsv in observables]

In [ ]:
!pip install cvxpy

In [ ]:
import cvxpy as cp

In [ ]:
def optimize(dim, obsv, expct, tol=1e-5):
    sigma = cp.Variable((dim, dim), complex=True)
    objective = cp.Minimize(cp.abs(cp.norm(sigma, 'nuc')))
    constraints = [cp.real(cp.trace(sigma)) == 1]
    for o, e in zip(obsv, expct):
        constraints.append(cp.abs(cp.trace(sigma @ Pauli(o).to_matrix()) - e) <= tol)
    problem = cp.Problem(objective, constraints)
    problem.solve()
    print(problem.status)
    return sigma.value
    
    
sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import eigh
import winsound

def optimize(dim, obsv, expct, tol=1e-3):
    # Initial guess for sigma (identity matrix scaled to satisfy the trace constraint)
    sigma_init = np.eye(dim) / dim

    # Objective function: minimize the nuclear norm of sigma
    def objective(sigma_flat):
        sigma = sigma_flat.reshape((dim, dim))
        # Compute the singular values (eigenvalues for symmetric matrices)
        _, s, _ = np.linalg.svd(sigma)
        return np.sum(s)

    # Constraint: trace(sigma) == 1
    def trace_constraint(sigma_flat):
        sigma = sigma_flat.reshape((dim, dim))
        return np.trace(sigma) - 1

    # Constraints: |trace(sigma @ Pauli(o)) - e| <= tol
    def pauli_constraints(sigma_flat):
        sigma = sigma_flat.reshape((dim, dim))
        constraints = []
        for o, e in zip(obsv, expct):
            pauli_matrix = Pauli(o).to_matrix()
            trace_value = np.trace(sigma @ pauli_matrix).real
            constraints.append(trace_value - e - tol)
            constraints.append(-trace_value + e - tol)
        return constraints

    # Combine all constraints
    constraints = [{'type': 'eq', 'fun': trace_constraint}]
    for i in range(len(obsv)):
        constraints.append({
            'type': 'ineq',
            'fun': lambda sigma_flat, i=i: pauli_constraints(sigma_flat)[2*i]
        })
        constraints.append({
            'type': 'ineq',
            'fun': lambda sigma_flat, i=i: pauli_constraints(sigma_flat)[2*i + 1]
        })

    # Flatten the initial sigma for the optimizer
    sigma_init_flat = sigma_init.flatten()

    # Solve the optimization problem
    result = minimize(objective, sigma_init_flat, constraints=constraints, method='L-BFGS-B')

    # Reshape the result back to a matrix
    sigma_opt = result.x.reshape((dim, dim))

    print(result.message)
    return sigma_opt

# Example usage
sigma = optimize(d, observables, expectations)
print(f'fidelity between outcome and target is {get_fidelity(sigma, state)}')

winsound.Beep(1000, 3000)


In [ ]:
np.trace(sigma)

In [ ]:
print(sigma_opt)

In [ ]:
np.real(np.trace(state)) == 1

In [ ]:
np.all(
    np.array(
        [np.abs(np.real(np.trace(state @ Pauli(o).to_matrix())) - e) <= 1e-3 for o, e in zip(observables, expectations)]
    )
)